# Agente Inteligente con LangGraph
Este notebook implementa un agente que decide automáticamente si debe buscar en documentos PDF o hacer una búsqueda web para responder preguntas.

In [58]:
# Celda 1: Configuración inicial
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from typing import TypedDict

load_dotenv()

api_key = os.getenv("API_KEY")
api_key_tavily = os.getenv("TAVILY_API_KEY")
model = ChatOpenAI(model="gpt-3.5-turbo", api_key=api_key, temperature=0)

In [59]:
# Celda 2: Clase para búsqueda en PDFs
from langchain.document_loaders import PyMuPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

class BusquedaPDF:
    def __init__(self, pdf_paths):
        docs = []
        for path in pdf_paths:
            loader = PyMuPDFLoader(path)
            docs.extend(loader.load())

        splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
        docs_fragmentados = splitter.split_documents(docs)

        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        self.vectorstore = FAISS.from_documents(docs_fragmentados, self.embeddings)

        llm = ChatOpenAI(model_name="gpt-3.5-turbo", api_key=api_key)
        self.qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=self.vectorstore.as_retriever())

    def run(self, query: str) -> str:
        respuesta = self.qa_chain.run(query)
        return respuesta

rutas_pdfs = ["Investigación de WindSurf.pdf", "nke-10k-2023.pdf"]
buscador_pdf = BusquedaPDF(rutas_pdfs)

In [60]:
# Celda 3: Herramienta de búsqueda web
from langchain_tavily import TavilySearch
from langchain.tools import Tool

tavily_tool = TavilySearch(max_results=5, topic="general")

def busqueda_internet(query: str) -> str:
    output = tavily_tool.invoke({"query": query})
    resultados = output.get("results", [])
    if not resultados:
        return "No se encontraron resultados relevantes en la búsqueda en línea."
    contenido = "\n\n".join([res["content"] for res in resultados[:3] if "content" in res])
    return contenido

tool_pdf = Tool(
    name="busqueda_pdf",
    func=buscador_pdf.run,
    description="Busca información en los documentos PDF cargados."
)

tool_web = Tool(
    name="busqueda_internet",
    func=busqueda_internet,
    description="Realiza búsquedas en Internet para información actualizada."
)

In [71]:
# Celda 4 Modificada: Configuración corregida de LangGraph
from langgraph.graph import StateGraph
from langchain.schema import SystemMessage, HumanMessage

class AgentState(TypedDict):
    input: str
    tool_used: str
    output: str
    next_step: str  # Nuevo campo para control de flujo

def decide_tool(state: AgentState) -> AgentState:
    pregunta = state["input"]
    prompt = [
        SystemMessage(content="Eres un asistente que elige la mejor herramienta según la consulta."),
        HumanMessage(content=f'''
Dada la siguiente pregunta: "{pregunta}", ¿cuál herramienta debería usar?

**NOTA**
-Siempre que hables de Windsurf me tienes que leer el pdf

Responde solo con el nombre exacto de la herramienta.
''')
    ]
    respuesta = model(prompt).content.strip().lower()
    # Retornamos un NUEVO ESTADO indicando el próximo paso
    if "pdf" in respuesta:
        return {"next_step": "usar_pdf"}
    elif "internet" in respuesta:
        return {"next_step": "usar_web"}
    else:
        return {"next_step": "fin"}  # Siempre retornar dict

def usar_pdf(state: AgentState) -> AgentState:
    query = state["input"]
    resultado = tool_pdf.run(query)
    return {"input": query, "tool_used": "busqueda_pdf", "output": resultado}

def usar_web(state: AgentState) -> AgentState:
    query = state["input"]
    resultado = tool_web.run(query)
    return {"input": query, "tool_used": "busqueda_internet", "output": resultado}

# Configuración CORREGIDA del grafo
graph = StateGraph(AgentState)
graph.add_node("decision", decide_tool)
graph.add_node("usar_pdf", usar_pdf)
graph.add_node("usar_web", usar_web)
graph.add_node("fin", lambda state: state)

graph.set_entry_point("decision")

# Configuración condicional corregida
graph.add_conditional_edges(
    "decision",
    lambda state: state.get("next_step", "fin"),  # Usamos el campo del estado
    {
        "usar_pdf": "usar_pdf",
        "usar_web": "usar_web",
        "fin": "fin"
    }
)

graph.add_edge("usar_pdf", "fin")
graph.add_edge("usar_web", "fin")

agent_executor = graph.compile()

In [72]:
# Celda 2 Modificada - Verificación de PDFs
class BusquedaPDF:
    def __init__(self, pdf_paths):
        try:
            print(f"🔍 Cargando PDFs desde: {pdf_paths}")
            docs = []
            for path in pdf_paths:
                if not os.path.exists(path):
                    raise FileNotFoundError(f"Archivo no encontrado: {path}")
                loader = PyMuPDFLoader(path)
                docs.extend(loader.load())
                print(f"✅ {path} cargado correctamente ({len(docs)} documentos)")
            
            splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
            docs_fragmentados = splitter.split_documents(docs)
            print(f"📚 Total fragmentos creados: {len(docs_fragmentados)}")

            self.embeddings = HuggingFaceEmbeddings(
                model_name="sentence-transformers/all-MiniLM-L6-v2",
                model_kwargs={'device': 'cpu'}  # Añadir parámetro para CPU
            )
            
            self.vectorstore = FAISS.from_documents(docs_fragmentados, self.embeddings)
            print("🦄 Vectorstore FAISS creado exitosamente")
            
            self.qa_chain = RetrievalQA.from_chain_type(
                llm=model,
                chain_type="stuff",
                retriever=self.vectorstore.as_retriever(search_kwargs={"k": 3}),
                return_source_documents=True
            )
            
        except Exception as e:
            print(f"❌ Error crítico al cargar PDFs: {str(e)}")
            raise

    def run(self, query: str) -> str:
        try:
            print(f"\n🔎 Buscando en PDFs: '{query}'")
            result = self.qa_chain.invoke({"query": query})
            
            if not result["source_documents"]:
                return "No se encontró información relevante en los documentos."
                
            sources = "\n".join(
                f"- {doc.metadata['source']} (página {doc.metadata.get('page', 'N/A')})"
                for doc in result["source_documents"]
            )
            
            return f"{result['result']}\n\nFuentes:\n{sources}"
            
        except Exception as e:
            print(f"⚠️ Error en búsqueda PDF: {str(e)}")
            return "Error al consultar los documentos."

In [73]:
# Celda 5: Chatbot interactivo
print("Chatbot de LangGraph - escriba 'salir' para terminar.")

while True:
    pregunta = input("Usuario: ")
    
    if pregunta.lower() in ["salir", "exit", "quit"]:
        print("Fin de la conversación.")
        break
    
    entrada_agente = {"input": pregunta, "tool_used": "none", "output": ""}
    resultado = agent_executor.invoke(entrada_agente)
    
    print("\nRespuesta:", resultado["output"])
    print("Herramienta usada:", resultado["tool_used"])
    print("-" * 40)
   

Chatbot de LangGraph - escriba 'salir' para terminar.


C:\Users\6003465\AppData\Local\Temp\ipykernel_31288\3478153105.py:25: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  respuesta = self.qa_chain.run(query)



Respuesta: Windsurf no parece ser un programa o término técnico conocido en el contexto proporcionado. Puede que se refiera a otro tipo de actividad o concepto fuera del contexto presentado. Si deseas más información o aclaración sobre Windsurf en otro contexto, por favor proporciona más detalles.
Herramienta usada: busqueda_pdf
----------------------------------------

Respuesta: How can I help you today?
Herramienta usada: busqueda_pdf
----------------------------------------

Respuesta: Las características clave de Windsurf son que utiliza Flows que combinan Agentes y Copilots para mejorar el flujo de desarrollo del programador, permite acceder al proyecto completo sin necesidad de especificaciones detalladas, lo que facilita generar código mientras se entiende el contexto. Además, Windsurf integra un lenguaje natural avanzado para un procesamiento avanzado.
Herramienta usada: busqueda_pdf
----------------------------------------

Respuesta: Júpiter es el planeta más grande del sis